# Use the results from magnet training to compute TPR, FPR and AUC

In [1]:
import numpy as np
from scipy.stats import entropy
from scipy import ndimage

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [3]:
def train_fs(distances, train_fpr):
    selected_distance_idx = int(np.ceil(len(distances) * (1-train_fpr)))
    threshold = sorted(distances)[selected_distance_idx-1]
    threshold = threshold
    #print ("Threshold value: %f" % threshold)
    return threshold

In [4]:
def test(model, dataset, X, threshold):
    distances = get_distance_test(model, dataset, X)
    Y_pred = distances > threshold
    return Y_pred, distances

In [5]:
def compute_distance(model, dataset, X):
    distances = get_distance_test(model, dataset, X)
    return distances

In [6]:
def get_tpr_fpr(true_labels, pred_labels):
    TP = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
    FP = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))

    AP = np.sum(true_labels)
    AN = np.sum(1-true_labels)

    tpr = TP/AP if AP>0 else np.nan
    fpr = FP/AN if AN>0 else np.nan

    return tpr, fpr, TP, AP, FP, AN

In [7]:
def evaluate_test(model, x_test, threshold, dataset):
    Y_all = np.concatenate([np.ones(len(x_test), dtype=bool)])
    #print(Y_all)
    Y_all_pred, Y_all_pred_score = test(model, dataset, x_test, threshold)
    tpr, fpr, tp, ap, fp, an = get_tpr_fpr(Y_all, Y_all_pred)
    return tpr
                                           

# Imagenet thresholds

In [47]:
import pandas as pd
import numpy as np

In [48]:
#train data
df_cifar = pd.read_csv("imagenet/Benign_IMAGENET_Metrics.csv")

In [49]:
recon_error_benign = df_cifar.iloc[0].values.flatten().tolist()[1:]

In [50]:
min(recon_error_benign), max(recon_error_benign)

(0.0256082750856876, 0.9809719324111938)

In [51]:
fpr = [0.01, 0.05, 0.1] 
for train_fpr in fpr: 
    print(train_fs(recon_error_benign, train_fpr))

0.7556061744689941
0.6121619939804077
0.5275646448135376


In [52]:
threshold = [0.75560617446899415, 0.6121619939804077, 0.5275646448135376]

# now compute TPR

In [53]:
def compute_tpr(adv, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in adv: 
        if value>threshold:
            TP += 1
        else: 
            FN += 1
    
    
    return (TP/(TP+FN))*100

In [54]:
#pgd with five epsilons
df_pgd_eps1 = pd.read_csv('imagenet/pgd/eps8_255.csv')
df_pgd_eps2 = pd.read_csv('imagenet/pgd/eps16_255.csv')
df_pgd_eps3 = pd.read_csv('imagenet/pgd/eps32_255.csv')
df_pgd_eps4 = pd.read_csv('imagenet/pgd/eps64_255.csv')


#fgsm with five epsilons
df_fgsm_eps1 = pd.read_csv('imagenet/fgsm/eps8_255.csv')
df_fgsm_eps2 = pd.read_csv('imagenet/fgsm/eps16_255.csv')
df_fgsm_eps3 = pd.read_csv('imagenet/fgsm/eps32_255.csv')
df_fgsm_eps4 = pd.read_csv('imagenet/fgsm/eps64_255.csv')

#bim with five epsilons
df_bim_eps1 = pd.read_csv('imagenet/bim/eps8_255.csv')
df_bim_eps2 = pd.read_csv('imagenet/bim/eps16_255.csv')
df_bim_eps3 = pd.read_csv('imagenet/bim/eps32_255.csv')
df_bim_eps4 = pd.read_csv('imagenet/bim/eps64_255.csv')

#cw with four confidences
df_cw_conf1 = pd.read_csv('imagenet/cw/conf_0.csv')
df_cw_conf2 = pd.read_csv('imagenet/cw/conf_50.csv')

In [55]:
magnet1 = df_pgd_eps1.iloc[0].values.flatten().tolist()[1:]
magnet2 = df_pgd_eps2.iloc[0].values.flatten().tolist()[1:]
magnet3 = df_pgd_eps3.iloc[0].values.flatten().tolist()[1:]
magnet4 = df_pgd_eps4.iloc[0].values.flatten().tolist()[1:]

In [56]:
for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    TPR = compute_tpr(magnet3, th)
    print(TPR)
    TPR = compute_tpr(magnet4, th)
    print(TPR)
    print('---')

2.3928215353938187
2.3976023976023977
2.0937188434695915
1.998001998001998
---
6.0817547357926225
6.093906093906094
5.583250249252243
5.294705294705294
---
11.06679960119641
10.789210789210788
10.767696909272182
10.589410589410589
---


In [57]:
magnet1 = df_fgsm_eps1.iloc[0].values.flatten().tolist()[1:]
magnet2 = df_fgsm_eps2.iloc[0].values.flatten().tolist()[1:]
magnet3 = df_fgsm_eps3.iloc[0].values.flatten().tolist()[1:]
magnet4 = df_fgsm_eps4.iloc[0].values.flatten().tolist()[1:]


for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    TPR = compute_tpr(magnet3, th)
    print(TPR)
    TPR = compute_tpr(magnet4, th)
    print(TPR)
    print('---')

1.1976047904191618
1.1964107676969093
1.0967098703888334
1.1940298507462688
---
4.990019960079841
5.18444666001994
5.383848454636091
5.771144278606965
---
9.980039920159681
10.16949152542373
10.767696909272182
12.736318407960198
---


In [58]:
magnet1 = df_bim_eps1.iloc[0].values.flatten().tolist()[1:]
magnet2 = df_bim_eps2.iloc[0].values.flatten().tolist()[1:]
magnet3 = df_bim_eps3.iloc[0].values.flatten().tolist()[1:]
magnet4 = df_fgsm_eps4.iloc[0].values.flatten().tolist()[1:]


for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    TPR = compute_tpr(magnet3, th)
    print(TPR)
    TPR = compute_tpr(magnet4, th)
    print(TPR)
    print('---')

0.4985044865403789
0.29940119760479045
0.09950248756218905
1.1940298507462688
---
2.8913260219341974
2.3952095808383236
1.2935323383084576
5.771144278606965
---
7.377866400797608
5.788423153692615
3.7810945273631837
12.736318407960198
---


In [59]:
magnet1 = df_cw_conf1.iloc[0].values.flatten().tolist()[1:]
magnet2 = df_cw_conf2.iloc[0].values.flatten().tolist()[1:]

for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    print('----')

0.6979062811565304
0.6979062811565304
----
1.8943170488534395
1.8943170488534395
----
3.988035892323031
3.988035892323031
----


# Check test FPR

In [60]:
df_cifar = pd.read_csv("imagenet/Test_Benign_IMAGENET_Metrics.csv")

In [61]:
df_cifar

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059
0,reconstruction error l1,0.267403,0.600209,0.518619,0.453901,0.533193,0.480909,0.154535,0.451119,0.398895,...,0.5772,0.047127,0.355181,0.054883,0.280507,0.159551,0.626549,0.192693,0.39751,0.373745


In [62]:
recon = df_cifar.iloc[0].values.flatten().tolist()[1:]
threshold = [0.75560617446899415, 0.6121619939804077, 0.5275646448135376]

In [63]:
def compute_FPR(ben, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in ben: 
        if value<threshold:
            TN += 1
        else: 
            FP += 1
    
    
    return (FP/(TN+FP))*100

In [64]:
for th in threshold:
    FPR = compute_FPR(recon, th)
    print(FPR)

1.6993464052287581
5.42483660130719
10.42483660130719


# Compute ROC AUC score

In [65]:
df_cifar = pd.read_csv("imagenet/Test_Benign_IMAGENET_Metrics.csv")
recon = df_cifar.iloc[0].values.flatten().tolist()[1:]

In [66]:
min(recon), max(recon)

(0.0211200155317783, 0.9809721112251282)

In [67]:
threshold = [0.98, 0.88, 0.80, 0.75560617446899415, 0.6121619939804077, 0.5275646448135376, 0.43, 0.33, 0.22, 0.12, 0.02]

In [68]:
fpr_results = []
for th in threshold:
    FPR = compute_FPR(recon, th)
    fpr_results.append(FPR/100)
print(fpr_results)

[0.00032679738562091496, 0.00196078431372549, 0.010130718954248365, 0.01699346405228758, 0.0542483660130719, 0.1042483660130719, 0.2352941176470588, 0.4473856209150327, 0.7382352941176471, 0.9297385620915033, 1.0]


In [69]:
#pgd with five epsilons
df_pgd_eps1 = pd.read_csv('imagenet/pgd/eps8_255.csv')
df_pgd_eps2 = pd.read_csv('imagenet/pgd/eps16_255.csv')
df_pgd_eps3 = pd.read_csv('imagenet/pgd/eps32_255.csv')
df_pgd_eps4 = pd.read_csv('imagenet/pgd/eps64_255.csv')


#fgsm with five epsilons
df_fgsm_eps1 = pd.read_csv('imagenet/fgsm/eps8_255.csv')
df_fgsm_eps2 = pd.read_csv('imagenet/fgsm/eps16_255.csv')
df_fgsm_eps3 = pd.read_csv('imagenet/fgsm/eps32_255.csv')
df_fgsm_eps4 = pd.read_csv('imagenet/fgsm/eps64_255.csv')

#bim with five epsilons
df_bim_eps1 = pd.read_csv('imagenet/bim/eps8_255.csv')
df_bim_eps2 = pd.read_csv('imagenet/bim/eps16_255.csv')
df_bim_eps3 = pd.read_csv('imagenet/bim/eps32_255.csv')
df_bim_eps4 = pd.read_csv('imagenet/bim/eps64_255.csv')

#cw with four confidences
df_cw_conf1 = pd.read_csv('imagenet/cw/conf_0.csv')
df_cw_conf2 = pd.read_csv('imagenet/cw/conf_50.csv')

In [70]:
def compute_TPR(adv, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in adv: 
        if value>threshold:
            TP += 1
        else: 
            FN += 1
    
    
    return (TP/(TP+FN))*100

In [71]:
def compute_auc(adv_list, th): 
    tpr_results = []
    for th in threshold:
        TPR = compute_TPR(adv_list, th)
        tpr_results.append(TPR/100)
    return sklearn.metrics.auc(fpr_results, tpr_results)

In [72]:
import sklearn
from sklearn.metrics import roc_auc_score

feature_squeezer_1 = df_pgd_eps1.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_2 = df_pgd_eps2.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_3 = df_pgd_eps3.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_4 = df_pgd_eps4.iloc[0].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))
print(compute_auc(feature_squeezer_3, threshold))
print(compute_auc(feature_squeezer_4, threshold))

0.5118981291419858
0.5199493317140376
0.5270005343446784
0.5609449374155256


In [73]:
feature_squeezer_1 = df_fgsm_eps1.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_2 = df_fgsm_eps2.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_3 = df_fgsm_eps3.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_4 = df_fgsm_eps4.iloc[0].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))
print(compute_auc(feature_squeezer_3, threshold))
print(compute_auc(feature_squeezer_4, threshold))

0.4997699046351741
0.5076518809584318
0.5318707276862225
0.6174753682567554


In [74]:
feature_squeezer_1 = df_bim_eps1.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_2 = df_bim_eps2.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_3 = df_bim_eps3.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_4 = df_bim_eps4.iloc[0].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))
print(compute_auc(feature_squeezer_3, threshold))
print(compute_auc(feature_squeezer_4, threshold))

0.4522290970226575
0.43037226201192386
0.38574691899977237
0.30069623466916645


In [75]:
feature_squeezer_1 = df_cw_conf1.iloc[0].values.flatten().tolist()[1:]
feature_squeezer_2 = df_cw_conf2.iloc[0].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))

0.16764510390397433
0.16764510390397433
